In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

In [184]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [259]:
import re

cond = re.compile('[가-힣]+')

def get_data(gallery):
  result = []
  for name in os.listdir(path='/content/drive/MyDrive/jolnon/' + gallery):
    with open('/content/drive/MyDrive/jolnon/' + gallery + '/' + name, 'r', encoding='utf-8') as f:
      data = json.load(f)
      valid = []
      for d in data['content'].split('\n'):
        res = []
        for word in d.split(' '):
          if cond.match(word) != None:
            res.append(word)
        d = ' '.join(res)
        if d and not d.replace(' ', '').isdecimal():
          valid.append(d)
      result.append('\n'.join([data['title']] + valid))
  return result

In [260]:
import os
import json
import re
from zipfile import ZipFile

cond = re.compile('[가-힣]+')

def get_data_from_zip(zip_name):
    zip_file_path = '/content/drive/MyDrive/jolnon/' + zip_name + '.zip'
    extract_path = '/content/' + zip_name

    # zip 파일 압축 해제
    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    result = []
    for name in os.listdir(path=extract_path):
        file_path = os.path.join(extract_path, name)
        if os.path.isdir(file_path):
            for name in os.listdir(path=file_path):
                with open(file_path + '/' + name, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    valid = []
                    for d in data['content'].split('\n'):
                        res = []
                        for word in d.split(' '):
                            if cond.match(word) != None:
                                res.append(word)
                        d = ' '.join(res)
                        if d and not d.replace(' ', '').isdecimal():
                            valid.append(d)
                    result.append('\n'.join([data['title']] + valid))
        else:
            with open(extract_path + '/' + name, 'r', encoding='utf-8') as f:
                data = json.load(f)
                valid = []
                for d in data['content'].split('\n'):
                    res = []
                    for word in d.split(' '):
                        if cond.match(word) != None:
                            res.append(word)
                    d = ' '.join(res)
                    if d and not d.replace(' ', '').isdecimal():
                        valid.append(d)
                result.append('\n'.join([data['title']] + valid))
    return result


In [261]:
import random
random.seed(42)
galleries = ['giants_new2','cock_tail']#,'ktwiz','kartriderdrift','skwyverns_new1','ncdinos','samsunglions_new','doosanbears_new1','giants_new2','sh_new','lgtwins_new','tigers_new']
dataset = []
for e in galleries:
  dataset.append([get_data_from_zip(e)])
cocktail = []
for data in dataset:
  for sen in data:
    cocktail+=sen

In [262]:
def remove_words(sentence, word):
    return ' '.join([w for w in sentence.split(' ') if word not in w.lower()])
stopwords = []#['ㅋㅋ', 'ex', '나는', 'on', '이미지', '순서', '오늘', '일단', 'and', '이야', '그리고', '내일', '그냥', '000', '조금', '살짝', 'ㅇㅇ', 'ㅈㄱㄴ', '있음', '이거', '내가', '칵하하하', '칵하핫', '근데', '지듣노', 'youtube ', '야스중', '우흥', '한다', 'ㅎㅎ', 'ㅠㅠ', '로오오오오오옹', '하고', '아침', '것도', '추천', '혹시', '새낀데', '같다']
for i in range(len(cocktail)):
  for j in range(len(stopwords)):
    cocktail[i] = remove_words(cocktail[i],stopwords[j])

In [263]:
import collections

def remove_top_n_words(data, n):
    words = [word for sentence in data for word in sentence.split(' ')]
    word_count = collections.Counter(words)
    top_n_words = [word for word, _ in word_count.most_common(n)]
    for i in range(len(data)):
        for word in top_n_words:
            if word=='':
                continue
            data[i] = remove_words(data[i], word)
    return data

def remove_empty_sentences(cocktail, real_labels):
  indices = [i for i, sentence in enumerate(cocktail) if sentence.strip() != '']
  cocktail = [cocktail[i] for i in indices]
  real_labels = [real_labels[i] for i in indices]
  return cocktail, real_labels


In [264]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i][0])
cocktail = remove_top_n_words(cocktail, 0)
cocktail, real_labels = remove_empty_sentences(cocktail, real_labels)

In [265]:
from transformers import AutoModel, AutoTokenizer
from umap import UMAP

# umap_model = UMAP(random_state=42)
umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', random_state=42)
model = AutoModel.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [266]:
# bertopic_model = BERTopic(language='multilingual',
#                           umap_model=umap_model,
#                           nr_topics=100,
#                           top_n_words=1)
# topics, probs = bertopic_model.fit_transform(cocktail)

In [267]:
# bertopic_model.visualize_topics()
# bertopic_model.visualize_distribution(probs[0])

In [268]:
# from transformers import RobertaModel, RobertaTokenizer
# import torch

# model = RobertaModel.from_pretrained("klue/roberta-large")
# tokenizer = RobertaTokenizer.from_pretrained("klue/roberta-large")

In [277]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  def __call__(self, target):
    return self.tokenizer.tokenize(target)
  # def __call__(self, target):
  #   tokens = self.tokenizer.tokenize(target)
  #   if isinstance(tokens[0], str):
  #     indices = [i for i, token in enumerate(tokens) if not token.startswith('##')]
  #     tokens = [tokens[i] for i in indices]
  #   elif isinstance(tokens[0][0], str):
  #     for j in range(len(tokens)):
  #       indices = [i for i, token in enumerate(tokens[j]) if not token.startswith('##')]
  #       tokens[j] = [tokens[j][i] for i in indices]
  #   return tokens

In [278]:
from sklearn.feature_extraction.text import TfidfVectorizer

custom_tokenizer = CustomTokenizer(tokenizer)
# vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [279]:
# bertopic_model_embedding = BERTopic(embedding_model=model,
#                                     umap_model=umap_model,
#                                     nr_topics=100,
#                                     top_n_words=1)
# topics, probs = bertopic_model_embedding.fit_transform(cocktail)

In [280]:
# bertopic_model_tokenizer = BERTopic(vectorizer_model=vectorizer,
#                                     umap_model=umap_model,
#                                     nr_topics=100,
#                                     top_n_words=1)
# topics, _ = bertopic_model_tokenizer.fit_transform(cocktail)

In [281]:
bertopic_model_embed_token = BERTopic(embedding_model=model,
                                      umap_model=umap_model,
                                      vectorizer_model=vectorizer,
                                      nr_topics=100,
                                      top_n_words=3)
topics, _ = bertopic_model_embed_token.fit_transform(cocktail)

In [282]:
topic_info = bertopic_model_embed_token.get_topic_info()
number_of_topics = len(topic_info) - 1
print(f"생성된 주제의 수: {number_of_topics}")
lst = []
for index, row in topic_info.iterrows():
    topic_num = row['Topic']
    if topic_num != -1:
        topic_name = row['Name']
        pure_topic_name = "_".join(topic_name.split("_")[1:])
        lst.append(pure_topic_name)
for i in lst:
  print(i)

생성된 주제의 수: 52
##찌_딱_뿌
##내_##받_##술
+_❤_!
##❤_##맙_##갓
##ㅏ_##ㅣ_##ㄱ
##g_j_##p
c_##u_앱
##싸_##쓰_##쏘
vs_##하하_칵
9_##단_##개
쓱_싸_##끄
봐라_병신_이거
7_5_##월
ㅋㅋ_##빈_##휴
가방_친구_##생
나부_하자_시작
나라_바닥_##액
쥐_##준_)
끊_##쪼_##ㅋㅋㅋㅋㅋ
이미지_순서_내릴
##내_##받_에반
롬_~_최강
?_##많_온
나눔_##릴_받
릴_이미지_순서
릴_'_릴레이
할머니_존나_오늘
소다_베르_##뭇
송도_인천_##국제
._##는_##도
나눔_##릴_만
나눔_##릴_받
##ㅅ_##어먹_붙
##모리_단_목
어제_##자_최강
경상도_베일_응원
털어_,_##을
입_##입_압
아이스_왔_##간
!_워_~
아무_뭐_쥬
소금_생겼_병
올라온_이야기_한다
견과_##박_용병
##는_##는데_.
운동_여자_걸
##는_##면_선물
합격_팝업_##렵
그런데_##짓_##딱
##이너_##스트_꽂
##카_##키_##칵
안치_##용_응


In [ ]:
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
bertopic_model_embed_token = BERTopic(embedding_model=model,
                                      umap_model=umap_model,
                                      vectorizer_model=vectorizer,
                                      ctfidf_model=ctfidf_model,
                                      nr_topics=100,
                                      top_n_words=3,
                                      calculate_probabilities=True)
topics, _ = bertopic_model_embed_token.fit_transform(cocktail)

In [ ]:
topic_info = bertopic_model_embed_token.get_topic_info()
number_of_topics = len(topic_info) - 1
print(f"생성된 주제의 수: {number_of_topics}")
lst = []
for index, row in topic_info.iterrows():
    topic_num = row['Topic']
    if topic_num != -1:
        topic_name = row['Name']
        pure_topic_name = "_".join(topic_name.split("_")[1:])
        lst.append(pure_topic_name)
for i in lst:
  print(i)